In [ ]:
from src.data_loader import load_all_data
from src.entity_resolution import build_entity_map
from src.data_linking import link_datasets
from src.timeline import generate_entity_timeline
from src.anomaly_detection import detect_anomalies
from src.predictive_monitoring import predict_next_location, predict_inactivity_alert

In [ ]:
# 1. Load Data
datasets = load_all_data()
profiles = datasets['profiles']

In [ ]:
# 2. Entity Resolution
entity_map = build_entity_map(profiles)

In [ ]:
# 3. Link Datasets
linked = link_datasets(datasets, entity_map)

In [ ]:
# Model Train
from src.timeline import generate_entity_timeline
from src.predictive_monitoring import train_markov_model

# Build timelines for many users to train Markov model
all_timelines = []
for e_id in profiles['entity_id'].head(500):   # sample first 500 users for speed
    tl = generate_entity_timeline(e_id, linked)
    if len(tl) > 1:
        all_timelines.append(tl)

markov_model = train_markov_model(all_timelines)
print("Markov model trained on", len(all_timelines), "timelines")

In [ ]:
# 4. Timeline for sample user
entity_id = profiles.iloc[0]['entity_id']
timeline = generate_entity_timeline(entity_id, linked)

In [ ]:
# 5. Anomaly detection
anomalies = detect_anomalies(entity_id, timeline)

In [ ]:
# 6. Predictions
pred, explain = predict_next_location(timeline)
status, reason = predict_inactivity_alert(timeline)

In [ ]:
print("Predicted next:", pred, "\nReason:", explain)
print("Inactivity alert:", status, "\nWhy:", reason)

In [ ]:
 # Test for one user
from src.predictive_monitoring import predict_next_location, predict_inactivity_alert

entity_id = profiles.iloc[0]['entity_id']
timeline = generate_entity_timeline(entity_id, linked)

pred, explain = predict_next_location(timeline, model=markov_model)
print("Predicted next:", pred)
print("Why:", explain)

status, reason = predict_inactivity_alert(timeline, model=markov_model)
print("Inactivity alert:", status)
print("Reason:", reason)